
## 🧠 LangChain — Foco em Componentes
LangChain é um framework de orquestração para construir agentes e pipelines com LLMs. Ele oferece uma série de abstrações reutilizáveis, como:

LLMs, Prompts, Chains, Tools, Agents, Memory, Retrievers

Ele funciona de forma sequencial (ex: LLMChain) ou com agentes que escolhem o próximo passo.

✅ Ideal para fluxos lineares ou com lógica baseada em linguagem.

## 🔄 LangGraph — Foco em Fluxos de Controle (Graph-Based)
LangGraph é uma biblioteca construída sobre o LangChain para criar fluxos de execução baseados em grafos — ou seja, você modela sua aplicação como nós (nodes) e arestas (edges), que representam a lógica de decisão.

✅ Ideal para construir máquinas de estado, agentes multi-etapas, workflows complexos, retries, loops, branching, etc.

Conceitos principais:
Node = uma função (Runnable) que processa um estado, nós

Edge = transição condicional entre nós, arestas(realiza conexões entre os nós)

Graph = define como os nós se conectam

State = objeto que carrega informações do processo em andamento

In [ ]:
%run ../helpers/00-llm.ipynb

In [ ]:
from helpers.llm import initialize_llm, logger, pretty_print
 
llm, _, _ = initialize_llm()

In [ ]:
from langgraph.graph import StateGraph, END
from IPython.display import Image, display
from langchain_core.runnables import RunnableLambda
from typing import TypedDict
from langchain.prompts import ChatPromptTemplate
from duckduckgo_search import DDGS

# Estado

In [ ]:
class State(TypedDict):
    pergunta: str  
    conteudo: str 
    resposta: str 

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
 

client = MultiServerMCPClient(
    {
        "weather": {
            # Ensure you start your weather server on port 8000
            "url": "http://127.0.0.1:8000/mcp",
            "transport": "streamable_http",
        },
        "pokemon": {
            # Ensure you start your weather server on port 8080
            "url": "http://127.0.0.1:8080/mcp",
            "transport": "streamable_http",
        },
          "math": {
            "command": "python",
            # Make sure to update to the full absolute path to your math_server.py file
            "args": ["tools/math_server.py"],
            "transport": "stdio",
        },
    }
)
tools = await client.get_tools()

In [ ]:
tools = await client.get_tools()
agent = create_react_agent(
    llm,
    tools
)
# math_response = await agent.ainvoke(
#     {"messages": [{"role": "user", "content": "what's (3 + 5) x 12?"}]}
# )
# print(math_response['messages'][-1].content)
# weather_response = await agent.ainvoke(
#     {"messages": [{"role": "user", "content": "what is the weather in nyc?"}]}
# )

# print(weather_response['messages'][-1].content)
 

pokemon_response = await agent.ainvoke(
    {"messages": [{"role": "user", "content": "Ash"}]}
)

print(pokemon_response['messages'][-1].content)

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.prebuilt import ToolNode, tools_condition

def call_model(state: MessagesState):
    response = llm.bind_tools(tools).invoke(state["messages"])
    # Suponha que `response` seja o objeto retornado pelo LLM (ChatMessage)
    tool_calls = response.additional_kwargs.get("tool_calls", [])

    # Itera sobre as chamadas de ferramenta e extrai o nome
    for call in tool_calls:
        tool_name = call.get("function", {}).get("name")
        print(f"Tool chamada: {tool_name}")
    if not tool_calls:
        print(f"Usuário perguntou: {state['messages'][-1].content}")     
    return {"messages": response}

builder = StateGraph(MessagesState)
builder.add_node(call_model)
builder.add_node(ToolNode(tools))


builder.add_edge(START, "call_model")
builder.add_conditional_edges(
    "call_model",
    tools_condition,
)
builder.add_edge("tools", "call_model")
graph = builder.compile()



In [ ]:
math_response = await graph.ainvoke({"messages": "me de informações sobre pikachu"})

print(math_response['messages'][-1].content)

In [ ]:
# Pergunta mista, primeiro resolve a soma utilizando IA depois a multiplicação usando o mcp multiplicador
math_response = await graph.ainvoke({"messages": "quanto é (3 + 5) x 12?"})

print(math_response['messages'][-1].content)

In [ ]:

weather_response = await graph.ainvoke({"messages": "what is the weather in nyc?"},config=config)

print(weather_response['messages'][-1].content)

In [ ]:
# Exibindo o grafo
display(Image(graph.get_graph().draw_mermaid_png()))

print(graph.get_graph().print_ascii())

In [ ]:
from langgraph.checkpoint.memory import MemorySaver, InMemorySaver
graph = builder.compile(checkpointer=InMemorySaver())

config={"configurable":{"thread_id":159}}

In [ ]:
#math_response = await graph.ainvoke({"messages": "what's (3 + 5) x 12?"})
weather_response = await graph.ainvoke({"messages": "what is the weather in nyc?"},config=config)

print(weather_response['messages'][-1].content)

In [ ]:
weather_response = await graph.ainvoke({"messages": "How is the weather in NYC?"},config=config)

print(weather_response['messages'][-1].content)

# Funções de CallBack

In [ ]:
def recebe_pergunta(state: State) -> State:
    print(f"Usuário perguntou: {state['pergunta']}")
    return {"pergunta": state["pergunta"]}

def precisa_pesquisar(state: State) -> State:
    pergunta = state["pergunta"].lower()
    precisa = any(p in pergunta for p in ["tempo","clima", "weather"])
    print("Precisa pesquisar?", precisa)
    # Retorna um dicionário, com chave especial para decisão
    return {"next_step": "pesquisar" if precisa else "consultar_llm"}

def pesquisar(state: State) -> State:
    pergunta = state["pergunta"]
    print(f"Pesquisando no DuckDuckGo: {pergunta}")

    with DDGS() as ddgs:
        resultados = ddgs.text(pergunta, max_results=1)

    if resultados:
        contexto = "\n".join([r["body"] for r in resultados if "body" in r])
    else:
        contexto = "Nenhum resultado encontrado."

    return {"conteudo": contexto}    

def consultar_llm(state: State) -> State:
    prompt = ChatPromptTemplate.from_template("Responda à seguinte pergunta: {pergunta}")
    chain = prompt | llm
    resposta = chain.invoke({"pergunta": state["pergunta"]})
    print("Resposta direta do LLM.")
    return {"resposta": resposta.content}

def sintetizar(state: State) -> State:
    contexto = state.get("conteudo", "")
    pergunta = state["pergunta"]
    prompt = ChatPromptTemplate.from_template("""
    Use o seguinte contexto para responder a pergunta:
    Contexto: {contexto}
    Pergunta: {pergunta}
    Resposta:""")
    chain = prompt | llm
    resposta = chain.invoke({"contexto": contexto, "pergunta": pergunta})
    print("Resposta sintetizada com contexto.")
    return {"resposta": resposta.content}

def responder(state: State) -> State:
    print("\n Resposta Final:")
    print(state["resposta"])
    return state

# Cria Grafo

In [ ]:
graph = StateGraph(State)

graph.add_node("recebe_pergunta", RunnableLambda(recebe_pergunta))
graph.add_node("decisao", RunnableLambda(precisa_pesquisar))
graph.add_node("pesquisar", RunnableLambda(pesquisar))
graph.add_node("consultar_llm", RunnableLambda(consultar_llm))
graph.add_node("sintetizar", RunnableLambda(sintetizar))
graph.add_node("responder", RunnableLambda(responder))

# Transições de Estado e Condições

In [ ]:
graph.set_entry_point("recebe_pergunta")

graph.add_edge("recebe_pergunta", "decisao")
graph.add_conditional_edges(
    "decisao",
    lambda state: state["next_step"],  
    {
        "pesquisar": "pesquisar",
        "consultar_llm": "consultar_llm"
    }
)
graph.add_edge("pesquisar", "sintetizar")
graph.add_edge("consultar_llm", "responder")
graph.add_edge("sintetizar", "responder")
graph.set_finish_point("responder")

# Execução

In [ ]:
executable = graph.compile()

print("\n TESTE 1:")
executable.invoke({"pergunta": "Qual é a capital da Alemanha?"})

print("\n  TESTE 2:")
executable.invoke({"pergunta": "Me mostre dados sobre economia brasileira em 2025."})

In [ ]:
# Exibindo o grafo
display(Image(executable.get_graph().draw_mermaid_png()))

print(executable.get_graph().print_ascii())

## Exemplo 3

In [ ]:
 
class ChatState(TypedDict):
    pergunta: str
    resposta: str

# Função que responde perguntas
def responder(state: ChatState) -> ChatState:
    pergunta = state.get("pergunta", "")
    resposta = llm.invoke(pergunta)
    return {"pergunta": pergunta, "resposta": resposta.content}

# Construindo o grafo
builder = StateGraph(ChatState)
 
builder.add_node("responder", responder)

builder.set_entry_point("responder")
builder.add_edge("responder", END)




In [ ]:
# Compilando o grafo
graph = builder.compile()
 

In [ ]:
# Exibindo o grafo
display(Image(graph.get_graph().draw_mermaid_png()))

print(graph.get_graph().print_ascii())


In [ ]:
pergunta = "Qual a capital do Brasil?"

# Criando o agente
entrada = ChatState({"pergunta": pergunta})
resultado = graph.invoke(entrada)

print(resultado)